<a href="https://colab.research.google.com/github/jpwhalley/GMS_Stats_Course/blob/master/6_Machine_Learning_Applications/NeuralNetworkTutorial_GMS2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Neural Network tutorial

In this tutorial we will be exploring a number of topics discussed in this morning's lecture, using a famous dataset, the MNIST collection of 70000 handwritten and labeled digits.  This will hopefully make these topics "come alive" by showing how they work in practice in a simple but realistic setting.

This tutorial makes use of a bunch of useful tools and frameworks: Keras, TensorFlow, Tensorboard, Numpy, Matplotlib, Python, jupyter, Google colab.  You can run this tutorial even if you haven't had previous experience with some or any of these.  Hopefully this tutorial gives you a starting point for exploring them further; in my experience they can massively boost your productivity in data exploration and research generally, and machine learning in particular.

## Initialization

This just loads the various packages we'll be using, and the MNIST data set.  

Make sure you run this on a GPU backend (runs faster) - go to Edit > Notebook settings or Runtime > Change runtime type and select GPU as Hardware accelerator. If you want to double-check, uncomment the line with `device_lib.list_local_devices()` and check that the output shows `device_type GPU` somewhere.

In [1]:
%tensorflow_version 1.x

from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

from tensorflow.keras import datasets
from tensorflow.keras.callbacks import TensorBoard

#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

dataformat="channels_last"
shape=[28,28,1]

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
print("Training: images",x_train.shape, " labels", y_train.shape, "; Test set",x_test.shape, " labels",y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
Training: images (60000, 28, 28)  labels (60000,) ; Test set (10000, 28, 28)  labels (10000,)


Always good practice to have a look at your data.  Here are the first two data points, plotted as a matrix, and their labels.

The data are just `numpy` arrays; their dimensions are printed above.  Visualisation is done using the `matplotlib` library.

*  Find a way to plot several digits with their labels in a grid.

In [0]:
plt.matshow(x_train[0])
plt.matshow(x_train[1])
print(y_train[0:2])

## Model 1: multinomial logistic regression

Below is the first complete model to predict the label (0 to 9) from the pixel intensities, in the simplest possible way (kindof) - using a multinomial logistic regression model, basically logistic regression but for classification instead of binary outcomes.  

(The `Sequential` refers to the fact that in this model (and all models in this tutorial), data flows sequentially through a number of layers from input to output.  This is not true for e.g. the Inception module, but Tensorflow/Keras also easily copy with such models.)

In detail, for an input $x \in R^{28\times 28} = R^{784}$, a vector representing the input image, the model predicts 

$$\hat{y} = A x,$$ 

where $A \in R^{10 \times 784}$ and $\hat{y}\in R^{10}$.  These are transformed into probabilities using the softmax function:

$$p(\mathrm{digit}=i) = { \exp(\hat{y}_i) \over \sum_j \exp(\hat{y}_j) }$$

To compare predicted probabilities $p$ for a given input $x_j$ with the actual class $y_j$, we use the `sparse_categorical_entropy` loss function.  (Here, `sparse` refers to the class encoding we use; we encode classes as integers 0 to 9, rather than "one-hot encoding", where we would represent e.g. class 2 as the vector $[0,1,0,0,0,0,0,0,0,0,0]$.)

For this model there is theory about how to obtain optimal parameters $A$ for a given data set; but in the spirit of neural networks, we are using a standard stochastic gradient descend algorithm, `adam`.  We will very quickly leave theory behind when we add even a little more complexity to the model, and then numerical optimization is the only option.

The call to `compile` builds a computational graph of the model.  The computation performed by the graph includes:

1.  evaluation of the model's prediction $F_\theta(x)$ over a batch $\{x_1,\ldots,x_B\}$; 
2. calculation of the loss $L_i := L( F_\theta(x_i), y_i )$;
3. calculation of the gradient $\nabla_\theta \sum_{i=1}^L L_i$, and
4. a single step of the optimization algorithm `adam`, resulting in new parameters $\theta$, ready to be executed on a GPU once you feed it data.  

If you've ever any of those steps by hand, you will appreciate how transformative Tensorflow/Keras (and similar frameworks) have been in machine learning research.

The call to `fit` finally runs the computational graph on successive batches of data.  One complete feed-through of all data is called an `epoch` in machine learning parlance.  The `fit` function is also given test data -- this is not used for training, but to evaluate the model's performance after each epoch.

*  How many parameters does this model have?
*  Find out how to print a summary of the model

In [2]:
tf.reset_default_graph()
tf.keras.backend.clear_session()   ## not entirely sure why this is necessary

## a very simple model - multinomial logistic regression (= softmax activation) on all 28 x 28 pixels
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

## set up logging via tensorboard
logdir = "logs_model1/{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
print("Writing log files to ",logdir)
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)

## pull the model, loss function, optimizer, and output metrics into a single computational graph
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## run the computational graph 50 times on the entire training data, to fit the model parameters
model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),callbacks=[tensorboard])

## evaluate the final model on the test set
print("Test set performance:")
model.evaluate(x_test,y_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Writing log files to  logs_model1/20191202-145753
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 63us/sample - loss: 0.4756 - acc: 0.8755 - val_loss: 0.3092 - val_acc: 0.9136
Epoch 2/50
60000/60000 [==============================] - 3s 57us/sample - loss: 0.3043 - acc: 0.9146 - val_loss: 0.2844 - val_acc: 0.9222
Epoch 3/50
60000/60000 [==============================] - 3s 55us/sample - loss: 0.2840 - acc: 0.9210 - val_loss: 0.2728 - val_acc: 0.9237
Epoch 4/50
60000/60000 [==============================] - 3s 56us/sample - loss: 0.2732 - acc: 0.9232 - val_loss: 0.2697 - val_acc: 0.9256
Epoch 5/50
60000/60000 [==============================] - 3s 55us/sample - loss: 0.2666 - acc: 0.9255 - val_loss: 0.2657 - val_acc: 0.9265
Epoch 6/50
26496/60000 [============>.................] - ETA: 1s - loss: 0.2583 - acc: 0.9286

KeyboardInterrupt: ignored

## Tensorboard

The `tensorboard` "callback" in the code above is called by the `fit` function at the end of each epoch.  The callback writes some logging information in a directory, which we can explore later.

Tensorboard is a standalone program that visualises the log files produced by tensorflow.  Running the cell below starts the tensorboard program inside the notebook.  You just have to do this once and use it for all runs in this session. Within tensorboard you can select the log files you want to look at.

Once the run above has finished, start Tensorboard and have a look at the run.  Notice that the model has overtrained - the validation loss has reached a minimum at around the 10th epoch, and gradually increased after that, even as the training loss continued to decrease.  This behaviour is seen quite often (but more usually after many more epochs).  Early Stopping is a heuristic that identifies the validation loss minimum, and stops training there.

*  Click on the 'graph' tab.  Can you make sense of the computational graph? Try double-clicking on nodes.

In [0]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs_model1

## Model 2 - a two-layer neural network

The accuracy (about 90%) achieved by the first model is not bad, but not hugely impressive either.  Part of the reason is that the model is purely a linear model, without nonlinearities or interactions.  So let's add a second layer of neurons, with the standard (ReLU) activation.

*  Compare this model's performance with that of model 1
*  Also compare the extent of overtraining that has occurred.
*  How many parameters does this model have?
*  Try running with fewer or more than 50 nodes in the middle layer.  How does this affect the model's performance and degree of overtraining?

In [0]:
tf.reset_default_graph()
tf.keras.backend.clear_session()

## a very simple model - multinomial logistic regression (= softmax activation) on all 28 x 28 pixels
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(50,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

logdir = "logs_twolayers/{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=25,validation_data=(x_test,y_test),callbacks=[tensorboard])
print("Test set performance:")
model.evaluate(x_test,y_test)

In [0]:
## tensorboard does not seem to find new data in old log directories, so start a new instance
%tensorboard --logdir logs_twolayers

## Convolutional neural network

Adding a second layer of neurons did improve performance a bit, but we're a long way off state of art.  A key shortcoming of the 
models so far is that they treat pixels in isolation.  In particular the models are not constrained to be translation-symmetric: shifting the input image by one pixel to the right should not make a difference in the output.

One way to help the model is to use a convolutional layer, which re-uses the same "kernel" and applies it across the (2D) image.
The next model uses two 2D convolutions, each using 'same' padding which ensures the output resolution is the same as the input resolution.  Each 2D convolution is followed by a $2\times 2$ max pooling layer, reducing the resolution with a factor 2 in both dimensions.

The first convolutional layer has 32 kernels, so its output is of dimension $28\times 28\times 32$.  After max pooling this becomes $14 \times 14\times 32$.  The next convolutional layer has 64 kernels, and after max pooling the output dimension is $7\times 7\times 64$.  This is then followed by a $50$-channel fully connected layer, and finally a $10$-channel output layer.

*  How many parameters does this model have?

In [0]:
tf.reset_default_graph()
tf.keras.backend.clear_session()

## a simple convolutional model:
## - 2d convolutional layer with 32 5x5 kernels, and ReLU activation (here implemented as a separate layer)
## - 2d max pooling, input size 2x2, and stride 2 in both dimensions
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape(target_shape=shape, input_shape=[28,28]),
    tf.keras.layers.Conv2D(16, 5, padding='same', data_format=dataformat, activation='linear'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2),(2,2), padding='same', data_format=dataformat),
    tf.keras.layers.Conv2D(16, 5, padding='same', data_format=dataformat, activation='linear'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2),(2,2), padding='same', data_format=dataformat),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

logdir = "logs_twoconvolutions/{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),callbacks=[tensorboard])
print("Test set performance:")
model.evaluate(x_test,y_test)

## Dropout

The convolutional model is pretty good, but still suffers from overtraining.  The trailing loss has become very low, while the
test accuracy stabilises.

Dropout randomly removes neurons from the network (sets the corresponding output to 0).  The probability that a neuron
is dropped out is set by the user.  It can be shown that the
effect of this is (approximately) equivalent to putting a 
prior on the parameters, causing them to shrink, which reduces
overtraining.

*  Does this address overtraining?  Why is the prediction accuracy higher than the training accuracy?  (Prediction is deterministic; how is this achieved with dropout layers?)

In [0]:
tf.reset_default_graph()
tf.keras.backend.clear_session()

dataformat="channels_last"
shape=[28,28,1]

## a simple convolutional model:
## - 2d convolutional layer with 32 5x5 kernels, and ReLU activation (here implemented as a separate layer)
## - 2d max pooling, input size 2x2, and stride 2 in both dimensions
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape(target_shape=shape, input_shape=[28,28]),
    tf.keras.layers.Conv2D(16, 5, padding='same', data_format=dataformat, activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),(2,2), padding='same', data_format=dataformat),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(16, 5, padding='same', data_format=dataformat, activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),(2,2), padding='same', data_format=dataformat),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50,activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

logdir = "logs_twoconvolutions_dropout/{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test),callbacks=[tensorboard])
print("Test set performance:")
model.evaluate(x_test,y_test)

## Data augmentation

The first convolutional layer shares weights across positions of the image.  It detects features like edges and corners, and this
weight sharing encodes our intuition that these features can occur
anywhere in the image.

More broadly, our model should interpret images of digits in the same may, no matter where the digits occur within the image.  In other words it should be "invariant" with respect to translations.
A convolutional layer helps to achieve this - shifting the input by one pixel, causes the output of the convolutional layer to also shift by one pixel. (This is not "invariance", but rather "equivariance" - the output is transformed by an "equivalent" transformation.)  However, the other layers (e.g. the max pool layer) break the symmetry again -- shifting the image by one pixel to the left, will give an output at the max pool layer that does  not correspond in a simple way to the original output.

Data augmentation helps to make the model more symmetric, by giving it more "equivalent" input data points, and let the model learn the required symmetry.

Rotational symmetry is another symmetry of the model - digits should be interpreted the same way if they are rotated by a small angle.

In [0]:
def augment_data(dataset, dataset_labels, num_augmented_images=1):
  augmented_images = []
  augmented_image_labels = []
  dataset_with_colourchan = np.reshape(dataset, dataset.shape + (1,))
  for num in range (dataset.shape[0]):
      ## original image
      augmented_images.append(dataset[num])
      augmented_image_labels.append(dataset_labels[num])
      
      for i in range(num_augmented_images):
        ## shift images by up to 0.05*28 (~1) pixels in any direction
        augmented_image = tf.contrib.keras.preprocessing.image.random_shift(dataset_with_colourchan[num], 0.05, 0.05, 
                                                                            row_axis=0, col_axis=1, channel_axis=2)
        ## add augmented image, dropping the colour channel again
        augmented_images.append( augmented_image[:,:,0] )
        augmented_image_labels.append(dataset_labels[num])

        ## rotate images by up to 20 degrees
        augmented_image = tf.contrib.keras.preprocessing.image.random_rotation(dataset_with_colourchan[num], 20, 
                                                                               row_axis=0, col_axis=1, channel_axis=2)
        augmented_images.append( augmented_image[:,:,0] )
        augmented_image_labels.append(dataset_labels[num])


  return np.array(augmented_images), np.array(augmented_image_labels)

## This creates an augmented dataset in memory.  This can also be done on the fly
## using ImageDataGenerator, but it turns out that's quite slow in this case.
x_aug_train, y_aug_train = augment_data(x_train, y_train, 1)

## Have a look at the first digit and its transformations (translation and rotation)
plt.matshow(x_aug_train[0])
plt.matshow(x_aug_train[1])
plt.matshow(x_aug_train[2])
print(y_aug_train[0:3])

In [0]:
tf.reset_default_graph()
tf.keras.backend.clear_session()

## the same model as above
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape(target_shape=shape, input_shape=[28,28]),
    tf.keras.layers.Conv2D(16, 5, padding='same', data_format=dataformat, activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),(2,2), padding='same', data_format=dataformat),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(16, 5, padding='same', data_format=dataformat, activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),(2,2), padding='same', data_format=dataformat),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50,activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

logdir = "logs_data_augmentation/{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir, histogram_freq=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_aug_train,y_aug_train,epochs=20,batch_size=3*32,validation_data=(x_test,y_test),callbacks=[tensorboard])
print("Test set performance:")
model.evaluate(x_test,y_test)

## Next steps

The model we have now is quite good (I'm getting ~99.5% test accuracy), but there probably still is room for improvement.  It contains quite a few parameters we could tweak; the data augmentation could
be refined; we can change the parameters of the optimizer, or the
optimizer itself; and we can add or remove layers or other features
of the model.

This exercise is called "hyperparameter search".  Strategies here range from a simple grid search, to empirical modeling of the outcome as a function of the hyperparameters and directing the search in that way.

Hyperparameter search is best done on a server or cluster, so
we won't try this in this tutorial.

Other topics not covered are dilated networks and residual networks, which we have found helpful in modeling DNA.
However, hopefully the two tutorials in this section have
given you a number of entry points with which to start building
your own models.


